In [1]:
import glob, os
import pdfplumber
import torch
import spacy
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import numpy as np
import pandas as pd
import os

## Data Cleaning

In [14]:
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])


def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def clean(data):
    all_text1 = BeautifulSoup(data).get_text()
    all_text1 = decontracted(all_text1)
    #new_data1[sentance] = re.sub("\S*\d\S*", "", new_data1[sentance]).strip() #to remove nos
    #new_data1[sentance] = re.sub('[^A-Za-z]+', ' ', new_data1[sentance])
    all_text1 = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", all_text1) # emails
    all_text1 = re.sub(r'[^\w ]+', "", all_text)
    all_text1 = re.sub(r'[A-Za-z]+[\d]+[\w]*|[\d]+[A-Za-z]+[\w]*'," ",all_text1) # for alphanummerical
    all_text1 = re.sub(r'[_]'," ",all_text1) #remove underscore
    # https://gist.github.com/sebleier/554280
    all_text1 = ' '.join(e.lower() for e in all_text1.split() if e.lower() not in stopwords)
    all_text1.strip()
    return all_text1

## Conversion of PDFS into TEXT

In [40]:
path = "D:\\OneDrive_2022-01-12\\Annotated_ALLSFA_21\\Unseen pdf"
os.chdir(path)
filename=[]
new_data1 = []
for file in glob.glob("*.pdf"):
    os.chdir(path)
    print(file)
    all_text = " "
    with pdfplumber.open(file) as pdf:
            for pdf_page in pdf.pages:
                single_page_text = pdf_page.extract_text()
                if len(single_page_text) > 30:
                    all_text = all_text + single_page_text
                    all_text = clean(all_text)
                    file_temp = file.replace('.pdf','.txt')
                    with open(file_temp,'w',encoding='UTF-8') as f:
                        f.write(all_text)
                else:
                    pass
    new_data1.append(all_text)

SFA_ZRG Partners LLC_v0.1_LB_20211007 edited_LB_20211021  (Final - TKJ Accepted1).pdf
SUPP-00002602 - CURIOUS LION SFA SUPPLIER FRAMEWORK AGREEMENT MSA.pdf


In [13]:
import spacy
import pandas as pd
from spacy import displacy
import os
import re


def check_expiration_date(effective_date):
    match_temp = re.search('\d\d\d\d', effective_date)
    match_temp1=re.search('\d\d$',effective_date)
    if match_temp is not None:
        find_string = str(match_temp.group())
        replace_string = str(int(match_temp.group()) + 2)
        return effective_date.replace(find_string,replace_string)

    else:
        if match_temp1 is not None:
            find_string = str(match_temp1.group())
            replace_string = str(int(match_temp1.group()) + 2)
            return effective_date.replace(find_string, replace_string)

def check_parameter(dic,TERM_LENGTH_CHECK,EXPIRATION_DATE_CHECK):
    if TERM_LENGTH_CHECK == True:
        dic_temp = ["TERM_LENGTH", False]

        for lab1 in dic['label']:
            if lab1 == dic_temp[0]:
                dic_temp = ["TERM_LENGTH", True]
                break
        if dic_temp[1] == False:
            dic["tag"].append("2 years")
            dic["label"].append(dic_temp[0])
    if EXPIRATION_DATE_CHECK == True:
        dic_temp1 = ["EXPIRATION_DATE", False]

        for lab1 in dic['label']:
            if lab1 == dic_temp1[0]:
                dic_temp = ["TERM_LENGTH", True]
                break
        if dic_temp1[1] == False:
            for tag, label in zip(dic["tag"], dic['label']):
                if label == "EFFECTIVE_DATE":
                    dic["tag"].append(check_expiration_date(tag))
                    dic["label"].append(dic_temp1[0])

    return dic


doc = nlp_ner(textData)
nlp_ner.max_length = len(textData)+100
dic_checking = {"tag":[],"label":[]}
for token in doc.ents:
    dic_checking["tag"].append(token.text)
    dic_checking["label"].append(token.label_)
dic_checking = check_parameter(dic_checking,TERM_LENGTH_CHECK=True,EXPIRATION_DATE_CHECK=True)
#for tag,label in zip(dic_checking["tag"],dic_checking['label']):
    #print(str(tag)+"----------"+label)

NameError: name 'textData' is not defined

## Loading Model

In [5]:
os.chdir(r'D:\OneDrive_2022-01-12\training_data\sm_output_47')
nlp_ner = spacy.load(r".\model-best") #load the best model

D:\Users\A1058564\Anaconda3\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.2.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


## Results in csv

In [10]:
def create_reports_from_text(location, nlp):
    for file in [f for f in os.listdir(location) if f.lower().endswith('.txt')]:
        text_file_location="{}{}".format(location,file)
        with open(text_file_location,encoding='UTF-8') as f:
            data = f.read()
            nlp_ner.max_length = len(textData)+100
            dic_checking = {"tag":[],"label":[]}
            for token in doc.ents:
                
    # print(token.text+'----'+token.label_)
                dic_checking["tag"].append(token.text)
                dic_checking["label"].append(token.label_)
        #nlp.max_length = len(data)+100
        #doc = nlp(data)
        #dict1 = {'Label': [], 'Output': []}
        #for token in doc.ents:
            #dict1['Label'].append(token.label_)
            #dict1['Output'].append(token.text)
            dic_checking = check_parameter(dic_checking,TERM_LENGTH_CHECK=True,EXPIRATION_DATE_CHECK=True)
            df = pd.DataFrame(dic_checking)
            file_temp = file.replace('.txt','.csv')
            file_temp = "{}{}".format(location,file_temp)
            df.to_csv(file_temp)

In [11]:
create_reports_from_text('D:\\OneDrive_2022-01-12\\21_ALLSFA\\', nlp_ner)

NameError: name 'textData' is not defined

In [3]:
os.chdir(r'D:\OneDrive_2022-01-12\training_data\sm_output_47')
!spacy evaluate ./output/model-best ./test.spacy

2022-05-10 12:49:46.546892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-10 12:49:46.546943: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Usage: python -m spacy evaluate [OPTIONS] MODEL DATA_PATH
Try 'python -m spacy evaluate --help' for help.

Error: Invalid value for 'DATA_PATH': Path './test.spacy' does not exist.


In [226]:
conda install -c conda-forge spacy-model-en_core_web_sm

Solving environment: ...working... done

## Package Plan ##

  environment location: D:\Users\A1058564\Anaconda3

  added / updated specs:
    - spacy-model-en_core_web_sm


The following NEW packages will be INSTALLED:

  _anaconda_depends  pkgs/main/win-64::_anaconda_depends-2021.11-py39_0
  beautifulsoup4     conda-forge/noarch::beautifulsoup4-4.10.0-pyha770c72_0
  catalogue          conda-forge/win-64::catalogue-2.0.6-py39hcbf5309_1
  chardet            conda-forge/win-64::chardet-4.0.0-py39hcbf5309_2
  cymem              conda-forge/win-64::cymem-2.0.6-py39h415ef7b_2
  cython-blis        conda-forge/win-64::cython-blis-0.7.6-py39h5d4886f_0
  filelock           conda-forge/noarch::filelock-3.6.0-pyhd8ed1ab_0
  langcodes          conda-forge/noarch::langcodes-3.3.0-pyhd8ed1ab_0
  murmurhash         conda-forge/win-64::murmurhash-1.0.6-py39h415ef7b_2
  numpy-base         pkgs/main/win-64::numpy-base-1.20.3-py39hc2deb75_0
  pandas             conda-forge/win-64::pandas-1.4.1-py39h2e25


The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2021.11=py39_0
  - defaults/win-64::anaconda-client==1.9.0=py39haa95532_0
  - defaults/win-64::anaconda-navigator==2.1.1=py39_0
  - defaults/noarch::anaconda-project==0.10.1=pyhd3eb1b0_0
  - defaults/win-64::astroid==2.6.6=py39haa95532_0
  - defaults/win-64::astropy==4.3.1=py39hc7d831d_0
  - defaults/noarch::backports.functools_lru_cache==1.6.4=pyhd3eb1b0_0
  - defaults/win-64::bcrypt==3.2.0=py39h196d8e1_0
  - defaults/noarch::binaryornot==0.4.4=pyhd3eb1b0_1
  - defaults/win-64::bkcharts==0.2=py39haa95532_0
  - defaults/win-64::bokeh==2.4.1=py39haa95532_0
  - defaults/win-64::bottleneck==1.3.2=py39h7cc1a96_1
  - defaults/win-64::clyent==1.2.2=py39haa95532_1
  - defaults/win-64::conda==4.11.0=py39haa95532_0
  - defaults/win-64::conda-build==3.21.6=py39haa95532_0
  - defaults/noarch::conda-pack==0.6.0=pyhd3eb1b0_0
  - defaults/noa

  wheel              conda-forge/noarch::wheel-0.37.1-pyhd8ed1ab_0
  xlrd               conda-forge/noarch::xlrd-2.0.1-pyhd8ed1ab_3

The following packages will be UPDATED:

  certifi            pkgs/main::certifi-2021.10.8-py39haa9~ --> conda-forge::certifi-2021.10.8-py39hcbf5309_1

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.10.26~ --> conda-forge::ca-certificates-2021.10.8-h5b45459_0
  openssl              pkgs/main::openssl-1.1.1l-h2bbff1b_0 --> conda-forge::openssl-1.1.1l-h8ffe710_0

The following packages will be DOWNGRADED:

  anaconda                                   2021.11-py39_0 --> custom-py39_1


Preparing transaction: ...working... done
Verifying transaction: ...working... failed


  - defaults/noarch::imageio==2.9.0=pyhd3eb1b0_0
  - defaults/win-64::ipykernel==6.4.1=py39haa95532_1
  - defaults/win-64::ipython==7.29.0=py39hd4e2768_0
  - defaults/noarch::ipywidgets==7.6.5=pyhd3eb1b0_1
  - defaults/noarch::jinja2==2.11.3=pyhd3eb1b0_0
  - defaults/noarch::jinja2-time==0.2.0=pyhd3eb1b0_2
  - defaults/noarch::jsonschema==3.2.0=pyhd3eb1b0_2
  - defaults/win-64::jupyter==1.0.0=py39haa95532_7
  - defaults/noarch::jupyterlab==3.2.1=pyhd3eb1b0_1
  - defaults/noarch::jupyterlab_pygments==0.1.2=py_0
  - defaults/noarch::jupyterlab_server==2.8.2=pyhd3eb1b0_0
  - defaults/noarch::jupyter_console==6.4.0=pyhd3eb1b0_0
  - defaults/win-64::jupyter_server==1.4.1=py39haa95532_0
  - defaults/win-64::matplotlib==3.4.3=py39haa95532_0
  - defaults/win-64::matplotlib-base==3.4.3=py39h49ac443_0
  - defaults/win-64::mkl_fft==1.3.1=py39h277e83a_0
  - defaults/win-64::mkl_random==1.2.2=py39hf11a4ad_0
  - defaults/noarch::nbclassic==0.2.6=pyhd3eb1b0_0
  - defaults/noarch::nbclient==0.5.3=pyhd

In [1]:
import gpt_2_simple as gpt2

In [17]:
import shutil
import os
import numpy as np
import argparse

def get_files_from_folder(path):

    files = os.listdir(path)
    return np.asarray(files)

def main(path_to_data, path_to_test_data, train_ratio):
    # get dirs
    _, dirs, _ = next(os.walk(path_to_data))

    # calculates how many train data per class
    data_counter_per_class = np.zeros((len(dirs)))
    for i in range(len(dirs)):
        path = os.path.join(path_to_data, dirs[i])
        files = get_files_from_folder(path)
        data_counter_per_class[i] = len(files)
    test_counter = np.round(data_counter_per_class * (1 - train_ratio))

    # transfers files
    for i in range(len(dirs)):
        path_to_original = os.path.join(path_to_data, dirs[i])
        path_to_save = os.path.join(path_to_test_data, dirs[i])

        #creates dir
        if not os.path.exists(path_to_save):
            os.makedirs(path_to_save)
        files = get_files_from_folder(path_to_original)
        # moves data
        for j in range(int(test_counter[i])):
            dst = os.path.join(path_to_save, files[j])
            src = os.path.join(path_to_original, files[j])
            shutil.move(src, dst)


def parse_args():
    parser = argparse.ArgumentParser(description="Dataset divider")
    parser.add_argument("--data_path", required=True,
    help="Path to data")
    parser.add_argument("--test_data_path_to_save", required=True,
    help="Path to test data where to save")
    parser.add_argument("--train_ratio", required=True,
    help="Train ratio - 0.7 means splitting data in 70 % train and 30 % test")
    return parser.parse_args()

if __name__ == "__main__":
    args = parse_args()
    main(args.data_path, args.test_data_path_to_save, float(args.train_ratio))

usage: ipykernel_launcher.py [-h] --data_path DATA_PATH --test_data_path_to_save TEST_DATA_PATH_TO_SAVE --train_ratio
                             TRAIN_RATIO
ipykernel_launcher.py: error: the following arguments are required: --data_path, --test_data_path_to_save, --train_ratio


SystemExit: 2

In [14]:
# 1.Get a list of the files
# 2.Randomize the files
# 3.Split files into training and testing sets
# 4.Do the thing

In [15]:
# 1.Get a list of the files
import os

def get_file_list_from_dir(datadir):
    all_files = os.listdir(os.path.abspath(datadir))
    data_files = list(filter(lambda file: file.endswith('.json'), all_files))
    return data_files

In [16]:
# 2.Randomize the files
import random

def randomize_files(file_list):
    shuffled = random.sample(list_of_files, len(list_of_files))
    return shuffled

In [6]:
# 3.Split files into training and testing sets

from math import floor

def get_training_and_testing_sets(file_list):
    split = 0.7
    split_index = floor(len(file_list) * split)
    training = file_list[:split_index]
    testing = file_list[split_index:]
    return training, testing

In [7]:
def ml_function(datadir = 'D:\200_Annotated_Files\Consolidated'):
    
    list_of_files = get_file_list_from_dir(datadir)
    suffled_data = randomize_files(list_of_files)
    training, testing = get_training_and_testing_sets(suffled_data)

In [54]:
import spacy
from spacy.tokens import DocBin
nlp1 =  spacy.blank('en')  # load a new spacy model
nlp2 =  spacy.blank('en')
db1 = DocBin() # create a DocBin object
db2 = DocBin()

In [55]:
import random
import os
from math import floor

In [59]:
def get_file_list_from_dir(datadir):
    all_files = os.listdir(os.path.abspath(datadir))
    data_files = list(filter(lambda file: file.endswith('.json'), all_files))
    return data_files

def randomize_files(list_of_files):
    shuffled = random.sample(list_of_files, len(list_of_files))
    return shuffled

def get_training_and_testing_sets(file_list):
    split = 0.8
    split_index = floor(len(file_list) * split)
    training = file_list[:split_index]
    testing = file_list[split_index:]
    return training, testing

def ml_function(datadir='D:\\75\\'):
    list_of_files = get_file_list_from_dir(datadir)
    suffled_data = randomize_files(list_of_files)
    training, testing = get_training_and_testing_sets(suffled_data)

In [65]:
ml_function(datadir='D:\\75\\')

AttributeError: 'NoneType' object has no attribute 'get_file_list_from_dir'

In [9]:
class random:
    
    def __init__(self, datadir):
        self.datadir = datadir
        
    
    def get_file_list_from_dir(self):
        all_files = os.listdir(os.path.abspath(self.datadir))
        data_files = list(filter(lambda file: file.endswith('.json'), all_files))
        return data_files

    def randomize_files(self):
        shuffled = random.sample(list_of_files, len(list_of_files))
        return shuffled

    def get_training_and_testing_sets(self):
        split = 0.8
        split_index = floor(len(file_list) * split)
        training = file_list[:split_index]
        testing = file_list[split_index:]
        return training, testing

    def ml_function(self):
        list_of_files = get_file_list_from_dir(self.datadir)
        suffled_data = randomize_files(list_of_files)
        training, testing = get_training_and_testing_sets(suffled_data)
        print(training,testing)

In [10]:
obj =random('D:\\200_Annotated_Files\\Consolidated\\')

In [13]:
obj.ml_function()

AttributeError: type object 'random' has no attribute 'sample'

In [20]:
os.chdir('D:\113_files\1\')
for filename in training:
    f = open(filename,encoding='UTF-8')
    TRAIN_DATA = json.load(f)

    for text, annot in tqdm(TRAIN_DATA['annotations']):
        doc1 = nlp1.make_doc(text) 
        ents = []
        for start, end, label in annot["entities"]:
            span = doc1.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc1.ents = ents 
        db1.add(doc1)

os.chdir(r'D:\OneDrive_2022-01-12\training_data')
db1.to_disk("./train.spacy") # save the docbin object

NameError: name 'training' is not defined

In [235]:
os.chdir('D:\RANDOMSAMPLES')
for filename in testing:
    f = open(filename,encoding='UTF-8')
    TEST_DATA = json.load(f)

    for text, annot in tqdm(TEST_DATA['annotations']):
        doc2 = nlp2.make_doc(text) 
        ents = []
        for start, end, label in annot["entities"]:
            span = doc2.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc2.ents = ents 
        db2.add(doc2)

os.chdir(r'D:\OneDrive_2022-01-12\training_data')
db2.to_disk("./test.spacy") # save the docbin object

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.86it/s]


In [241]:
testing

['Sepire MSA FINAL.json',
 'Revelation Software_EULA & Schedules_033007.json',
 'Zelis Healthcare-Master Services and License Agreement-Master Service Agreement-06-26-2018 (1).json',
 'SUPP-00000785 - SOW- 09-23-2021 - 12-31-2021.json',
 "TRADUCTIONS SERGE BELAIR INC-Master Service Agreement-Master Services Agreement-5-01-2017'.json",
 'Premier International Enterprises, Inc.json',
 'ON24 INC MSA.json',
 'WILLIS TOWERS WATSON US LLC MSA.json',
 'NorthgateArinsoMSAandSOW_signed6-22-10.json',
 'MSA signature page.json',
 'Topic SRL - MSA.json',
 'WIPRO_MSA_2018-04-09_001.json',
 'NorthgateArinso - e-know - Web Services Agreement FULLY EXECUTED 9-30-10.json',
 'S-001338 - SFA - 09-10-2021 - 09-09-2023.json',
 'WORKFORCE OUTSOURCE SERVICES INC-Master Service Agreement-Consulting Agreement-10-25-2017.json',
 'SUNGARD AVAILABILITY SERVICES LP-Master Service Agreement-Subscription Services Agreement-7-01-2018.json',
 'WINDWARD STUDIOS INC-Master Software License Agreement-License Agreement-11

In [ ]:
import random

with open("datafile.txt", "rb") as f:
    data = f.read().split('\n')

random.shuffle(data)

train_data = data[:50]
test_data = data[50:]

In [66]:
os.chdir(r'D:\OneDrive_2022-01-12\training_data')
!spacy evaluate ./output/model-best ./test2.spacy

[i] Using CPU

================================== Results ==================================

TOK     100.00
NER P   53.68 
NER R   35.42 

2022-04-19 12:15:03.820698: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-19 12:15:03.820753: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
D:\Users\A1058564\Anaconda3\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.2.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)



NER F   42.68 
SPEED   17091 


=============================== NER (per type) ===============================

                               P       R       F
SUPPLIER_NAME              57.14   19.05   28.57
SUPPLIER_STREET_ADDRESS    57.14   30.77   40.00
NET_PAYMENT_TERM           75.00   81.82   78.26
NOTICE_PERIOD              37.04   50.00   42.55
ALIGHT_ENTITY              50.00   64.71   56.41
SUPPLIER_ZIP_CODE          50.00    7.69   13.33
SUPPLIER_CITY              60.00   21.43   31.58
SUPPLIER_STATE             25.00    9.09   13.33
TERM_LENGTH               100.00   42.86   60.00
EFFECTIVE_DATE            100.00   38.46   55.56
SUPPLIER_COUNTRY            0.00    0.00    0.00
SUPPLIER_STREET_ADRESS      0.00    0.00    0.00

